In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import Table
from sqlalchemy.engine import URL
from sqlalchemy import MetaData
metadata = MetaData()
import pandas as pd
import pyodbc
from getpass import getpass

#Connection Parameters
conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER=ge-prd.database.windows.net;
DATABASE=GreenEnergy_DBP;
UID=azureuser;
PWD=Sommar2025@@;
Encrypt=yes;
TrustServerCertificate=no;
"""

#Establish connection
connection_url = URL.create(
    "mssql+pyodbc",
    query={"odbc_connect": conn_str}
)
engine = create_engine(connection_url)


#Table Deletion
energy_table = Table("energy_data_STG", metadata, autoload_with=engine)
energy_table.drop(engine, checkfirst=True)
# Table Creation
create_query = """
CREATE TABLE [dbo].[energy_data_STG](
	[datetime] [datetime] NULL,
	[consumption_kwh] [float] NULL,
	[solar_generation_kwh] [float] NULL,
	[temperature_C] [float] NULL,
	[humidity_percent] [float] NULL,
	[wind_speed_kmph] [float] NULL,
	[ETL_LOAD_FLAG] [int] DEFAULT 0
)
"""
with engine.connect() as connection:
    connection.execute(text(create_query))  
    connection.commit() 

#Load data
df = pd.read_csv(r"C:\Users\chnds\Downloads\Capstoneproject_Phase2\merged_energy_data_cleaned.csv")
df.to_sql("energy_data_STG", con=engine, if_exists="append", index=False)

197